In [9]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np

In [11]:
kr6 = KR6()
mr.IKinSpace(kr6._Slist, kr6._M, np.array([[0, 1,  0,     -5],
                      [1, 0,  0,      4],
                      [0, 0, -1, 1.6858],
                      [0, 0,  0,      1]]), [0,0,0,0,0,0], 0.01, 0.01)

(array([ 100.16159124, -618.77855132, 1244.15738289, 3282.70749718,
         823.45523396, 1491.11752806]),
 False)